In [ ]:
%%bash
emle-stop
cd ADP_AMBER
# Specify names in topology and coordinate files.
PARM=adp.parm7
CRD=adp.rst7

# Remove and re-create the output directory.
rm -rf output
mkdir output

# Switch to the output directory.
cd output

export EMLE_QM_XYZ_FREQUENCY=100
export EMLE_QM_XYZ_FILE=qm.xyz
export EMLE_PC_XYZ_FILE=pc.xyz

# Launch the emle-server in the background. (Sander will connect to this via ORCA.)
emle-server > server_log.txt 2>&1 &

# Launch sander.
sander -O -i ../emle.in -o emle.out -p ../adp.parm7 -c ../adp.rst7 -r emle.ncrst -x emle.nc > store.txt 2>&1

# Stop any running emle-server processes.
emle-stop

cd ../..

In [ ]:
import math
import os
import sys

import numpy as np
import scipy.io
import matplotlib.pyplot as plt

import openmm
import openmm.unit as unit
from openmm.app import *
from openmm import CustomBondForce, CustomCVForce

from emle.calculator import EMLECalculator

import sire as sr

mols = sr.load_test_files("ala.crd", "ala.top")
f = sr.save(mols, "output", format=["PRM"])
prm = AmberPrmtopFile(f"ala.top")

d = mols.dynamics(
    timestep="1fs",
    constraint="none",
    platform="cpu",
)

d.minimise()

calculator = EMLECalculator(device="cpu")
qm_mols, engine = sr.qm.emle(mols, mols[0], calculator)

In [ ]:
d = qm_mols.dynamics(
    timestep="1fs",
    constraint="none",
    qm_engine=engine,
    platform="cpu",
)


In [ ]:
from copy import deepcopy
context = d.context()
omm_system = context.getSystem()
integrator = deepcopy(context.getIntegrator())

In [ ]:
new_context = openmm.Context(omm_system, integrator, context.getPlatform())
new_context.setPositions(context.getState(getPositions=True).getPositions())


In [ ]:
# Initial equilibrium.
integrator.step(50)

In [ ]:
# Production sampling.
file_handle = open(f"ADP_OpenMM/ADP_OpenMM.dcd", "bw")
dcd_file = DCDFile(file_handle, prm.topology, dt=integrator.getStepSize())
for x in range(100):
    integrator.step(100)
    state = new_context.getState(getPositions=True)
    positions = state.getPositions()
    dcd_file.writeModel(positions)
file_handle.close()

In [ ]:
traj = sr.load("ala.top", "ADP_OpenMM/ADP_OpenMM.dcd")
traj[0].trajectory().view()

In [ ]:
traj = sr.load("ala.top", "ADP_OpenMM/ADP_OpenMM.dcd")
traj.trajectory().view()

In [ ]:
%%bash
cd ADP_AMBER/output

emle-analyze --qm-xyz qm.xyz \
             --pc-xyz pc.xyz \
             --emle-model ../../ADP/generic/analyze_generic.mat \
             --orca-tarball ../../ADP/generic/generic.tar \
             --backend torchani \
             result.mat
